In [ ]:
!pip install timm filetype

     |████████████████████████████████| 225kB 6.9MB/s 


In [ ]:
import numpy as np 
import pandas as pd 
import shutil
import os
import zipfile
import torch
import torch.nn as nn
import cv2
import matplotlib.pyplot as plt
import torchvision
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms
import copy
import tqdm
from PIL import Image
import torch.nn.functional as F
import albumentations
import torchvision.datasets as dataset
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import natsort
%matplotlib inline
import time
from collections import defaultdict
import datetime
from sklearn.cluster import KMeans

In [ ]:
!nvidia-smi

Mon Sep  7 07:53:27 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!unzip '/content/drive/My Drive/data.zip'

In [ ]:
data_transforms = transforms.Compose([
    transforms.ToPILImage(),                            
    transforms.Resize((300,300)),                                  
    #transforms.RandomRotation(45),
    transforms.ToTensor()
    ])

class ALLDataset(Dataset):
    
    def __init__(self, root_dir,folder,img_list, transform=None):
        self.root_dir=root_dir
        self.img_list = img_list
        self.transform=transform
        
        self.folder=folder

    def __len__(self):
        return len(self.img_list)
    
    def __getitem__(self, idx):
        
        self.image_name = self.img_list[idx]
        image = cv2.imread(os.path.join(self.root_dir,self.folder,self.image_name))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
        if self.transform:
            image = self.transform(image)
      
        return image,self.image_name

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [ ]:
def train_model(model_conv):
    model_conv.to(device)
    model_conv.eval()
    submission=pd.DataFrame()
    root_dir='/content/dataset'
    folders=os.listdir(root_dir)
    i=1
    for folder in folders:
      print(i)
      i=i+1
      images=os.listdir(os.path.join(root_dir,folder))
      testset = ALLDataset(root_dir, folder, images, transform=data_transforms)
      test_load = torch.utils.data.DataLoader(testset,batch_size=10,num_workers = 4)
      for batch_i, (data,names) in enumerate(test_load):
        with torch.no_grad():
            start_time = datetime.datetime.now()
            data = data.to(device)
            output = model_conv(data)
            output=output.cpu().detach().numpy()
            print(output.shape)
            kmeans = KMeans(n_clusters=2, random_state=1).fit(output)
            end_time = datetime.datetime.now()
            time_diff = (end_time - start_time)
            execution_time = time_diff.total_seconds() * 1000
            labels=kmeans.labels_
            names=list(names)
            teams=dict(zip(names, labels))
            sep_teams = defaultdict(list)
            for key, val in sorted(teams.items()):
              sep_teams[val].append(key)
              teams_dict=dict(sep_teams)
            teams_dict['team_a'] = teams_dict.pop(0)
            teams_dict['team_b'] = teams_dict.pop(1)
            folder_row={folder:teams_dict}
            team_df=pd.DataFrame.from_dict(folder_row,orient='index')

            team_df['time (ms)']=execution_time

      submission=submission.append(team_df)
  
    submission.reset_index(level=0, inplace=True)
    submission=submission.rename(columns={"index": "id"})
    submission.team_a=submission.team_a.apply(lambda x: [i.split('.')[0] for i in x])
    submission.team_b=submission.team_b.apply(lambda x: [i.split('.')[0] for i in x])
    submission.team_a=submission.team_a.apply(lambda x: ','.join(x))
    submission.team_b=submission.team_b.apply(lambda x: ','.join(x))

    return submission


In [ ]:
#model = torchvision.models.mobilenet_v2(pretrained=False, progress=True)
from timm.models import efficientnet_b3
model = efficientnet_b3(pretrained=True)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b3_ra2-cf984f9c.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b3_ra2-cf984f9c.pth


In [ ]:
model

In [ ]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x


#model.dropout = Identity()
#model.fc=Identity()
model.classifier=Identity()
#model.classifier[1]=Identity()
#for m in model.modules():
  #if isinstance(m, nn.BatchNorm2d):
    #m.eval()
#model.AdaptiveAvgPool2d=Identity()

In [ ]:
sample_subm=train_model(model)

In [ ]:
#sample_subm.to_csv('/content/drive/My Drive/5_subm__efficient_net_bn_off.csv',index=False)

In [ ]:
sample_subm.shape

(2200, 4)